# Imports

In [1]:
from load_df import load_df, restrict_df

In [2]:
dfx = load_df("data/train/x_train.csv")
dfy = load_df("data/train/y_train.csv")
#df_test = load_df("data/x_test.csv")

In [3]:
#from filter import Filter
#filter = Filter()
#filter_information = filter.filter(dfx)

In [4]:
#s_idx = filter_information[~filter_information["value"]].index.tolist()
#dfx = dfx[dfx["obs_id"].isin(s_idx)]
#dfy = dfy[dfy["obs_id"].isin(s_idx)]

In [7]:
#dfx.reset_index(drop=True, inplace=True)

In [9]:
#BATCH = 10000
#dfx = restrict_df(df=dfx,
#                  start=0,
#                  end=BATCH)
#dfy = restrict_df(df=dfy,
#                  start=0,
#                  end=BATCH)

# Data preprocessing

In [10]:
CATEGORY_COLUMNS = ["venue","action","trade"]
SCALAR_COLUMNS = ["price","bid","ask","bid_size","ask_size","flux"]
INDEX_COLUMN= "obs_id"

In [11]:
from preprocessing import Preprocessor
preprocessor = Preprocessor()

In [12]:
trainx_array = preprocessor.input_df_to_array(df=dfx,
                         category_columns=CATEGORY_COLUMNS,
                         scalar_columns=SCALAR_COLUMNS,
                         index_column=INDEX_COLUMN)


In [13]:
print(preprocessor.columns)

['price', 'bid', 'ask', 'bid_size', 'ask_size', 'flux', 'obs_id', 'venue_0', 'venue_1', 'venue_2', 'venue_3', 'venue_4', 'venue_5', 'action_A', 'action_D', 'action_U', 'trade_False', 'trade_True']


In [14]:
prices = trainx_array[:,:,0]
bid = trainx_array[:,:,1]
ask = trainx_array[:,:,2]
bid_size = trainx_array[:,:,3]
ask_size = trainx_array[:,:,4]
flux = trainx_array[:,:,5]

In [15]:
import numpy as np
window_size = 11

def rolling_mean(arr):
    padded_arr = np.pad(arr, ((0, 0), (window_size//2, window_size//2)), mode='edge')
    rolling_mean = np.apply_along_axis(lambda x: np.convolve(x, np.ones(window_size)/window_size, mode='valid'),
                                        axis=1, arr=padded_arr)
    return rolling_mean

trainx_array[:,:,0] = rolling_mean(prices)
trainx_array[:,:,1] = rolling_mean(bid)
trainx_array[:,:,2] = rolling_mean(ask)
trainx_array[:,:,3] = rolling_mean(bid_size)
trainx_array[:,:,4] = rolling_mean(ask_size)
trainx_array[:,:,5] = rolling_mean(flux)

In [16]:
dfx

,obs_id,venue,order_id,action,side,price,bid,ask,bid_size,ask_size,trade,flux
0,1,4,0,A,B,-0.08,0.00,0.04,2,75,False,100
1,1,4,1,D,A,0.17,0.00,0.04,2,75,False,-20
2,1,4,2,A,A,0.20,0.00,0.04,2,75,False,100
3,1,0,3,A,A,0.15,0.00,0.04,2,75,False,100
4,1,0,4,A,A,0.15,0.00,0.04,2,75,False,100
...,...,...,...,...,...,...,...,...,...,...,...,...
606395,9996,1,51,D,A,0.06,-0.07,0.00,314,1080,False,-100
606396,9996,1,68,A,A,0.02,-0.07,0.00,314,1080,False,100
606397,9996,5,69,A,B,-0.47,-0.07,0.00,314,1080,False,100
606398,9996,1,70,A,A,0.19,-0.07,0.00,314,1080,False,100


In [17]:
#test_array = input_df_to_array(df=df_test,
#                         category_columns=CATEGORY_COLUMNS,
#                         scalar_columns=SCALAR_COLUMNS,
#                         index_column=INDEX_COLUMN)


In [18]:
trainy_array = preprocessor.output_df_to_array(df=dfy)

In [19]:
print(len(trainx_array))
print(len(trainy_array))

6064
6064


In [20]:
TT_SPLIT = 400
array_list = [trainx_array,trainy_array]
new_arrays = preprocessor.shuffle_2d_sections(array_list)
trainx_array, trainy_array = new_arrays[0] , new_arrays[1]
testx_array , testy_array = trainx_array[-TT_SPLIT:,:,:], trainy_array[-TT_SPLIT:,:]
trainx_array, trainy_array = trainx_array[:-TT_SPLIT,:,:], trainy_array[:-TT_SPLIT,:]

In [21]:
print(trainx_array.shape)

(5664, 100, 17)


# Model definition

In [22]:
from model import create_model

2024-03-30 23:58:14.683626: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 23:58:15.647405: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 23:58:15.659016: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-30 23:58:17.664192: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [23]:
N_TIMESTEPS, N_FEATURES, N_OUTPUTS = trainx_array.shape[1], trainx_array.shape[2], trainy_array.shape[1]
model = create_model(n_timesteps=N_TIMESTEPS,
                     n_features=N_FEATURES,
                     n_outputs=N_OUTPUTS)

ValueError: Input 0 of layer "lstm_1" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 100)

In [ ]:
VERBOSE, EPOCHS, BATCH_SIZE = 0, 50, 1000
model.fit(trainx_array,
          trainy_array,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          verbose=VERBOSE)

In [ ]:
prediction = model.predict(testx_array)
_, accuracy = model.evaluate(testx_array, testy_array, batch_size=BATCH_SIZE, verbose=0)

13/13 [==============================] - 1s 24ms/step


In [ ]:
print(prediction)

[[0.03361205 0.04899666 0.04434716 ... 0.04294426 0.04092692 0.04420273]
 [0.03361205 0.04899666 0.04434716 ... 0.04294426 0.04092692 0.04420273]
 [0.03361199 0.04899675 0.04434713 ... 0.04294429 0.04092687 0.04420279]
 ...
 [0.03361205 0.04899668 0.04434716 ... 0.04294427 0.04092692 0.04420274]
 [0.03361198 0.04899677 0.04434713 ... 0.04294429 0.04092686 0.0442028 ]
 [0.0632062  0.03617761 0.0395727  ... 0.04386102 0.0283412  0.03559653]]


In [ ]:
print(accuracy)

0.03999999910593033


In [ ]:
output = [max(enumerate(row), key=lambda x: x[1])[0] for row in prediction]

In [ ]:
print(output)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 20, 1, 1, 1, 1, 1, 1, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
#import pandas as pd
#output_df = pd.DataFrame({"obs_id":list(df_test["obs_id"].unique()),"eq_code_cat":output})

In [ ]:
#output_df.to_csv("data/submission_benchmark.csv",index=False)